# __Problem Solution-Usage of the Model__

## __Step-1: Generating Usable Dataset__

In [1]:
# Run this cell individually to see how we have obtained usable dataset
import pandas as pd

df1 = pd.read_csv("pre_processable_dataset.csv")
df2 = pd.read_csv("SDSS_DR18.csv")

# Making df2 preprocessable
df2 = pd.DataFrame({"alpha":df2["ra"], "delta":df2["dec"], "u":df2["u"], 
                        "g":df2["g"], "r":df2["r"], "i":df2["i"], "z":df2["z"],
                         "redshift":df2["redshift"], "class":df2["class"]})

print(df2.loc[df2["i"] == df2["i"].min(),["alpha","delta","i","z","class"]])
df2.drop(64000, inplace=True)        
df2.reset_index(drop=True, inplace=True)   

""" 
Merge Usage:
Give those rows of df2 which are not in df1
res = pd.merge(df2, df1, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

[A] Parameters Explained:

1. pd.merge(left, right): Merges left and right DataFrames. In this case:
left = df2
right = df1

2. how='outer': Determines the type of join, with these options:
'left': Includes all rows from left (df2) and matching rows from right (df1).
'right': Includes all rows from right (df1) and matching rows from left (df2).
'inner': Includes only rows with matching values in both DataFrames.
'outer': Includes all rows from both DataFrames, even if they don’t match, resulting in a union of the DataFrames.

3. indicator=True: Adds a column _merge that indicates the origin of each row:
left_only: The row is unique to df2.
right_only: The row is unique to df1.
both: The row exists in both DataFrames.

[B] Filtering Rows with .query and Dropping _merge:

1. .query('_merge=="left_only"') filters rows unique to df2 (where _merge is left_only).
2. .drop('_merge', axis=1) removes the _merge column from the final output.
"""
df3 = pd.merge(df2, df1, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
df3.reset_index(drop=True, inplace=True)
df3.to_csv("SDSS_DR18-SDSS_DR17.csv", index=False)
df3

            alpha     delta       i       z   class
64000  152.317433 -0.740319 -9999.0 -9999.0  GALAXY


,alpha,delta,u,g,r,i,z,redshift,class
0,0.003092,-9.222288,19.37950,18.01700,17.25675,16.84228,16.58519,0.133806,GALAXY
1,0.004481,15.333154,18.34867,17.46815,17.10681,16.96127,16.93186,-0.000613,STAR
2,0.013397,14.872887,19.53350,18.60320,18.25266,18.14965,18.14450,0.000087,STAR
3,0.013890,-10.721064,19.17472,17.30671,16.35667,15.81012,15.42653,0.082817,GALAXY
4,0.019155,-10.976304,19.03506,18.01470,17.52182,17.22950,17.13396,0.150205,GALAXY
...,...,...,...,...,...,...,...,...,...
97919,359.978339,14.502116,18.98773,17.42024,16.76743,16.37206,16.12952,0.089775,GALAXY
97920,359.979852,-10.447743,18.83053,17.75476,17.32202,16.93194,16.82578,0.091997,GALAXY
97921,359.993568,-9.100617,19.02678,17.43318,16.46105,15.98610,15.62527,0.151658,GALAXY
97922,359.994814,-9.692961,18.20756,17.08489,16.54569,16.17881,16.00229,0.075514,GALAXY


## __Step-2: Model Inaugration__

In [2]:
# For using model, change the index between (0, 97923)
from Outliers import remove_outliers_with_isolation_forest
from joblib import load
import pandas as pd

usable = pd.read_csv("SDSS_DR18-SDSS_DR17.csv")
pre_processor = load("pre_processor.joblib")
model = load("Model.joblib")

index = 9199           
features = usable.iloc[index:index+1].drop(columns=["class"], axis=1)   
features_pp_nda = pre_processor.transform(features)
features_pp_df = pd.DataFrame(features_pp_nda, columns=usable.columns[:8])
AEC_prediction = model.predict(features_pp_df)

print(f"The astro entity with the given celestial coordinates and spectral properties is {AEC_prediction[0]}")
print(f"\nActual Label: ")
usable.iloc[index:index+1]

The astro entity with the given celestial coordinates and spectral properties is GALAXY

Actual Label: 


,alpha,delta,u,g,r,i,z,redshift,class
9199,29.425488,13.390228,19.05368,18.16397,17.96573,17.79586,17.88527,0.043639,GALAXY
